# <img src ='https://airsblobstorage.blob.core.windows.net/airstream/Asset 275.png' width="50px"> A. Aggregations and Transformations

We'll take a look at aggregation operations that you can perform using Spark.

In [0]:
import pandas as pd

In [0]:
insurance_pandas_df = pd.read_csv('/dbfs/FileStore/PLSampleDataSets/insurance.csv')

insurance_df = spark.createDataFrame(insurance_pandas_df)

In [0]:
# GroupBy is again a wide transformation as can be seen from the first job computing on all 4 slots and then compiling the results of the shuffle writes.

# TODO Recording for the cell below:

# click the i next to the stage no. for the first job. 
# We will this time click on the Associated job id 12 (for me which will vary) which is in blue color. This opens up details for the job. 

# Here again we click on the associated SQL Query no. We can see both jobs 12 and 13 have succeded. 
# Scroll and show the query plan

# Select the check box to expand all the details in the query plan visualization. 
# Hover over each node of the query plan visualisation. the pop up along with the details explains what is happening under the hood. 

In [0]:
 insurance_df.groupBy('sex')\
             .count()\
             .display()

sex,count
female,662
male,676


In [0]:
# TODO Recording for cell below

# Expand the Spark Jobs
# Expand each Job and show the stages (do not need to click and open up monitoring UI)

In [0]:
# If we need to see the proportions instead of counts
from pyspark.sql.functions import round

gender_data_counts =  insurance_df.groupBy('sex')\
                                  .count()\
                                  .withColumnRenamed('count', 'total')

gender_data_proportions = gender_data_counts\
                              .withColumn('proportions', round(gender_data_counts.total/insurance_df.count() * 100, 2))\
                              .drop('total')\
                              .display()

sex,proportions
female,49.48
male,50.52


In [0]:
charges_by_smokinghabit = insurance_df.groupBy('smoker')\
                                      .agg({'charges': 'avg'})\
                                      .withColumnRenamed('avg(charges)', 'average_charges')

charges_by_smokinghabit.display()

smoker,average_charges
no,8434.2682978562
yes,32050.23183153284


In [0]:
# TODO Recording for cell below

# After showing the tabular result
# Click on the Bar Graph for visualization
# Select Plot Options
# Drag Smoker to the Keys
# Drag average_charges to the values
# Make sure bar graph is selected in the graph drop down
# Click apply and show the result as a bar graph

In [0]:
charges_by_smokinghabit = insurance_df.groupBy('smoker')\
                                      .agg({'charges': 'avg', 'bmi': 'average', 'sex': 'count'})\
                                      .withColumnRenamed('avg(charges)', 'average_charges')

charges_by_smokinghabit.display()

smoker,average_charges,count(sex),avg(bmi)
no,8434.2682978562,1064,30.651795112781954
yes,32050.23183153284,274,30.70844890510949


In [0]:
insurance_df.agg({'charges': 'sum'}).display()

sum(charges)
1.7755824990759E7


In [0]:
insurance_df.groupBy('region')\
            .agg({'charges': 'sum'})\
            .withColumnRenamed('sum(charges)', 'region_revenue')\
            .display()

region,region_revenue
northwest,4035711.9965399993
southeast,5363689.76329
northeast,4343668.583308999
southwest,4012754.6476200004


In [0]:
insurance_df.groupBy('region')\
            .agg({'charges':'sum'})\
            .withColumnRenamed('sum(charges)','region_revenue')\
            .orderBy('region_revenue')\
            .display()

region,region_revenue
southwest,4012754.6476200004
northwest,4035711.9965399993
northeast,4343668.583308999
southeast,5363689.76329


In [0]:
insurance_df.groupBy('region')\
            .agg({'charges': 'avg'})\
            .withColumnRenamed('avg(charges)', 'average_charges')\
            .orderBy('average_charges', ascending = False)\
            .display()

region,average_charges
southeast,14735.41143760989
northeast,13406.3845163858
northwest,12417.575373969228
southwest,12346.937377292308


In [0]:
insurance_df.groupBy('children')\
            .agg({'charges': 'avg', 'children': 'count'})\
            .withColumnRenamed('avg(charges)', 'average_charges')\
            .orderBy('average_charges')\
            .display()

children,average_charges,count(children)
5,8786.035247222222,18
0,12365.975601635884,574
1,12731.171831635804,324
4,13850.6563112,25
2,15073.563733958335,240
3,15355.318366815289,157


In [0]:
# The family with 5 children seems to have unusually low insurance cost. 
# Let's check why is it the case by zooming in to the data of only families with no. of children = 5

# Seems like most insured people with children = 5 don't smoke

In [0]:
children_5 = insurance_df.filter(insurance_df.children == 5)

children_5.display()

age,sex,bmi,children,smoker,region,charges,insuranceclaim
19,female,28.6,5,no,southwest,4687.797,No
31,male,28.5,5,no,northeast,6799.458,No
20,female,37.0,5,no,southwest,4830.63,No
25,male,23.9,5,no,southwest,5080.096,No
45,male,24.31,5,no,southeast,9788.8659,No
52,female,46.75,5,no,southeast,12592.5345,Yes
49,female,31.9,5,no,southwest,11552.904,No
33,male,42.4,5,no,southwest,6666.243,No
33,male,33.44,5,no,southeast,6653.7886,No
46,male,25.8,5,no,southwest,10096.97,Yes


In [0]:
# Let's write out the result to a csv file. go to data> create table> dbfs>filestore>output and you can see that the csv file has been successfully created
dbutils.fs.rm("dbfs:/FileStore/output/children5.csv", True)

children_5.write.csv("dbfs:/FileStore/output/children5.csv")

In [0]:
# Let's read the data and verify the written data.
csvDF = spark.read.format('csv')\
                  .option("inferSchema", 'true') \
                  .option("header", 'true')\
                  .load("dbfs:/FileStore/output/children5.csv")
csvDF.display()

39,female,18.3,5,yes,southwest,19023.26,No
41,male,29.64,5,no,northeast,9222.4026,Yes
39,female,23.87,5,no,southeast,8582.3023,No
28,male,24.3,5,no,southwest,5615.369000000001,No
43,male,25.52,5,no,southeast,14478.33015,Yes
45,male,24.31,5,no,southeast,9788.8659,No
52,female,46.75,5,no,southeast,12592.5345,Yes
49,female,31.9,5,no,southwest,11552.903999999999,No
33,male,42.4,5,no,southwest,6666.243,No
46,male,25.8,5,no,southwest,10096.97,Yes
39,female,24.225,5,no,northwest,8965.79575,No


In [0]:
# Let's use this notebook to write this children_are_5 dataframe to a json file using the notebook as a job run from cluster.

#=> go to the side bar and select job from the menu
#=> on the window that now opens click on the create Job button.
#=> give the Job a name "Module-02"
#=> in schedule type let the option be manual
#=> under task choose the notebook from users>cloud.user@loonycorn.com and Module2_clusterRun notebook
#=> Clicking 'Confirm' chooses the notebook for the job
#=> Under cluster choose the loony_cluster
#=> now clicking the 'Create' button on the top will create the job
#=> Proceed to the Jobs window from the sidebar again
#=> Now you can see "Module-02" as existing job. 
#=> press the play button to run the job.
#=> the job will take some time to run and after the run is complete the status under the Last Run turns to succeeded.

# After the job has succeeded click on the succeeded status to take you to the outputs of the run
# We can launch the spark UI by clicking on 'View spark UI' just above the 'Output' title
# Here we can see all the jobs that have run with the description of the task ie the first line of a command block of the notebook. At what time was the job run and how long did it take to run the job. how many stages were present in the job and how many succeeded. It also shows how many tasks were run
# click on the stages tab on top. here we are able to check out stage by stage overview. The additional info here is if the stage had an input or output operation involved and also if there was a shuffle read or write that had to be done. 
#  go to data> create table> dbfs>filestore>output and you can see that now the json file has been successfully created too.
dbutils.fs.rm("dbfs:/FileStore/output/children5.json", True)
children_are_5.write.json("dbfs:/FileStore/output/children5.json")